# Import Required Libraries
Import the necessary libraries, including requests and pandas.

In [1]:
# Importing the required libraries
import requests
import pandas as pd



# Define API Endpoint and Parameters
Define the iNaturalist API endpoint and the parameters for the GET request.

In [2]:
# Define the iNaturalist API endpoint and parameters
api_endpoint = "https://api.inaturalist.org/v1/observations"

# Define the parameters for the GET request
# Replace 'city_name' with the name of the city you are interested in
params = {
    'place_id': 'laval', 
    'verifiable': 'true',
    'order': 'desc',
    'order_by': 'created_at'
}

# Send GET Request to iNaturalist API
Use the requests library to send a GET request to the iNaturalist API.

In [3]:
from pyinaturalist import set_user_agent, create_observation

# Set your user agent to identify your application
set_user_agent('YourAppName', '1.0', 'your@email.com')

# Set your user agent to identify your application
set_user_agent('YourAppName', '1.0', 'your@email.com')

# Set your API token
api_token = 'your_api_token_here'

# Example observation data
observation_data = {
    'species_guess': 'Vulpes vulpes',
    'observed_on_string': '2024-03-09',
    'latitude': 40.7128,
    'longitude': -74.0060,
    'place_guess': 'New York, NY, USA',
    'description': 'A wild fox was seen in the park.',
}

# Create the observation
response = create_observation(observation_data, api_token=api_token)

# Check if the observation was created successfully
if response.get('id'):
    print(f"Observation created successfully with ID: {response['id']}")
else:
    print("Failed to create observation:", response)

# Extract the results from the JSON data
results = data['results']

# Convert the results to a pandas DataFrame
df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
df.to_csv('inaturalist_data.csv', index=False)

print("Data saved to 'inaturalist_data.csv'.")

GET request to iNaturalist API failed with status code: 500


KeyError: 'results'

# Parse API Response
Parse the JSON response from the API and convert it into a pandas DataFrame.

In [ ]:
# Parse the JSON response and convert it into a pandas DataFrame
# Extract the relevant information from each observation
observations = []
for result in results:
    observation = {
        'id': result['id'],
        'observed_on': result['observed_on'],
        'description': result['description'],
        'latitude': result['geojson']['coordinates'][1],
        'longitude': result['geojson']['coordinates'][0],
        'location': result['location'],
        'species_guess': result['species_guess']
    }
    observations.append(observation)

# Convert the list of observations into a pandas DataFrame
df = pd.DataFrame(observations)

# Save the DataFrame to a CSV file
df.to_csv('inaturalist_data.csv', index=False)

print("Data saved to 'inaturalist_data.csv'.")

# Filter Data by City
Filter the DataFrame based on the city or cities of interest.

In [ ]:
# Define the cities of interest
cities_of_interest = ['city1', 'city2', 'city3']  # replace with your cities

# Filter the DataFrame based on the city or cities of interest
df_filtered = df[df['location'].isin(cities_of_interest)]

# Save the filtered DataFrame to a CSV file
df_filtered.to_csv('filtered_inaturalist_data.csv', index=False)

print("Filtered data saved to 'filtered_inaturalist_data.csv'.")

# Save Data to CSV
Save the filtered data to a CSV file using pandas.

In [ ]:
# Save the filtered DataFrame to a CSV file
df_filtered.to_csv('filtered_inaturalist_data.csv', index=False)

print("Filtered data saved to 'filtered_inaturalist_data.csv'.")